#  Python  for Data Analysis v2 | Notes_ Chapter_8 数据规整：聚合、合并和重塑

 本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用 Python 进行数据分析·第2版》第8章 数据规整：聚合、合并和重塑](https://www.jianshu.com/p/cfc035bae567) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用    Python    进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：    Python     3.6 

---

# Chapter 8 数据规整：聚合、合并和重塑


在许多应用中，数据可能分散在许多文件或数据库中，存储的形式也不利于分析。本章关注可以**聚合、合并、重塑数据的方法。**

首先，我会介绍 pandas 的层次化索引，它广泛用于以上操作。然后，我深入介绍了一些特殊的数据操作。在第 14 章，你可以看到这些工具的多种应用。

## 8.1 层次化索引

层次化索引（hierarchical indexing）是 pandas 的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。我们先来看一个简单的例子：创建一个 Series，并用一个由列表或数组组成的列表作为索引：

```
In [3]: data = pd.Series(np.random.randn(9), 
             index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2, 3, 1, 3, 1, 2, 2, 3]] )

In [4]: data
Out[4]:
a  1    0.519274
   2    1.593852
   3   -0.627557
b  1   -0.022029
   3   -0.684012
c  1    0.101001
   2    0.146661
d  2    0.013020
   3    1.687800
dtype: float64

```
看到的结果是经过美化的带有  MultiIndex  索引的 Series 的格式。索引之间的“间隔”表示“直接使用上面的标签”：

```
In [5]: data.index
Out[5]:
MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1,
 2]])


```

对于一个层次化索引的对象，可以使用所谓的**部分索引**，使用它选取数据子集的操作更简单：


```
In [7]: data['c']
Out[7]:
1    0.101001
2    0.146661
dtype: float64

In [8]: data['c':'a']
Out[8]: Series([], dtype: float64)

In [9]: data['b':'d']
Out[9]:
b  1   -0.022029
   3   -0.684012
c  1    0.101001
   2    0.146661
d  2    0.013020
   3    1.687800
dtype: float64

In [10]: data.loc[['b', 'c']]
Out[10]:
b  1   -0.022029
   3   -0.684012
c  1    0.101001
   2    0.146661
dtype: float64

In [11]: data.loc[['a', 'd']]
Out[11]:
a  1    0.519274
   2    1.593852
   3   -0.627557
d  2    0.013020
   3    1.687800
dtype: float64


```
有时甚至还可以在“内层”中进行选取：

```
In [12]: data[:, 2]
Out[12]:
a    1.593852
c    0.146661
d    0.013020
dtype: float64

In [13]: data['a',2]
Out[13]: 1.5938517892256658


```
层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。例如，可以通过 unstack 方法将这段数据重新安排到一个 DataFrame 中：


```
In [14]: data.unstack()
Out[14]:
          1         2         3
a  0.519274  1.593852 -0.627557
b -0.022029       NaN -0.684012
c  0.101001  0.146661       NaN
d       NaN  0.013020  1.687800



```

unstack 的逆运算是stack：

```
In [15]: data.unstack().stack()
Out[15]:
a  1    0.519274
   2    1.593852
   3   -0.627557
b  1   -0.022029
   3   -0.684012
c  1    0.101001
   2    0.146661
d  2    0.013020
   3    1.687800
dtype: float64


In [16]: data.unstack().unstack()
Out[16]:
1  a    0.519274
   b   -0.022029
   c    0.101001
   d         NaN
2  a    1.593852
   b         NaN
   c    0.146661
   d    0.013020
3  a   -0.627557
   b   -0.684012
   c         NaN
   d    1.687800
dtype: float64

In [17]: data.unstack().unstack().unstack()
Out[17]:
          a         b         c        d
1  0.519274 -0.022029  0.101001      NaN
2  1.593852       NaN  0.146661  0.01302
3 -0.627557 -0.684012       NaN  1.68780


```

stack 和 unstack 将在本章后面详细讲解。

对于一个 DataFrame ，每条轴都可以有分层索引：

```
In [21]: frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
    ...:    ....:                      index=[['a', 'a', 'b', 'b'], [1,
    ...:  2, 1, 2]],
    ...:    ....:                      columns=[['Ohio', 'Ohio', 'Color
    ...: ado'],
    ...:    ....:                               ['Green', 'Red', 'Green
    ...: ']])
    ...:

In [22]: frame
Out[22]:
     Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

# 第一列 a, b 是索引 ,第二列  1，2，1,2 也是索引  后三列是数据
# 列名 和行名一样， ohio 上面显示的 代表 第一列 和第二列都是 Ohio ,第三列是 Colorado

In [23]: np.arange(12).reshape((4,3))
Out[23]:
array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [24]: frame.index
Out[24]:
MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])


```

各层都可以有名字（可以是字符串，也可以是别的 Python 对象）。如果指定了名称，它们就会显示在控制台输出中：


```
In [28]: frame.index.names = ['key1', 'key2']

In [29]: frame.columns.names = ['state', 'color']

In [30]: frame
Out[30]:
state      Ohio     Colorado
color     Green Red    Green
key1 key2
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [31]: frame.unstack()
Out[31]:
state  Ohio            Colorado
color Green    Red        Green
key2      1  2   1   2        1   2
key1
a         0  3   1   4        2   5
b         6  9   7  10        8  11

In [32]: frame.unstack().stack()
Out[32]:
state     Colorado  Ohio
color        Green Green Red
key1 key2
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10


```

>注意：小心区分索引名 state、color 与行标签。

有了部分列索引，因此可以轻松选取列分组：


```
In [36]: frame['Ohio']
Out[36]:
color      Green  Red
key1 key2
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [37]: frame['Ohio', 'Green']
Out[37]:
key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int32


```

可以单独创建 MultiIndex 然后复用。上面那个 DataFrame 中的（带有分级名称）列可以这样创建：


```

In [41]: pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Gre
    ...: en', 'Red', 'Green']], names=['state','color'])
Out[41]:
MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])


```
## 重排与分级排序

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel 接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）：

```
In [42]: frame.swaplevel('key1','key2')
Out[42]:
state      Ohio     Colorado
color     Green Red    Green
key2 key1
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

```

而 sort_index 则根据单个级别中的值对数据进行排序。交换级别时，常常也会用到 sort_index ，这样最终结果就是按照指定顺序进行字母排序了：

```
In [45]: frame.sort_index(level=1)
Out[45]:
state      Ohio     Colorado
color     Green Red    Green
key1 key2
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [46]: frame.swaplevel(0,1)
Out[46]:
state      Ohio     Colorado
color     Green Red    Green
key2 key1
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [47]: frame.swaplevel(0,1).sort_index(level=0)
Out[47]:
state      Ohio     Colorado
color     Green Red    Green
key2 key1
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11


```

## 根据级别汇总统计

许多对  DataFrame  和 Series 的描述和汇总统计都有一个 level 选项，它用于指定在某条轴上求和的级别。再以上面那个  DataFrame  为例，我们可以根据行或列上的级别来进行求和：

```
In [55]: frame.sum(level='key2')
Out[55]:
state  Ohio     Colorado
color Green Red    Green
key2
1         6   8       10
2        12  14       16

In [56]: frame.sum(level='color', axis=1)
Out[56]:
color      Green  Red
key1 key2
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10


```

这其实是利用了 pandas 的 groupby 功能，本书稍后将对其进行详细讲解。

## 使用 DataFrame 的列进行索引

人们经常想要将 DataFrame 的一个或多个列当做行索引来用，或者可能希望将行索引变成 DataFrame 的列。以下面这个 DataFrame 为例：

```
In [57]: frame = pd.DataFrame({'a':range(7), 'b':range(7,0,-1),'c':['on
    ...: e', 'one', 'one', 'two', 'two',
    ...:    ....:                             'two', 'two'],
    ...:    ....:                       'd': [0, 1, 2, 0, 1, 2, 3]})
    ...:

In [58]: frame
Out[58]:
   a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3



```
DataFrame 的 set_index 函数会将其一个或多个列转换为行索引，并创建一个新的 DataFrame ：

```
In [60]: frame2 = frame.set_index(['c', 'd'])

In [61]: frame2
Out[61]:
       a  b
c   d
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [62]: frame2.unstack()
Out[62]:
       a                   b
d      0    1    2    3    0    1    2    3
c
one  0.0  1.0  2.0  NaN  7.0  6.0  5.0  NaN
two  3.0  4.0  5.0  6.0  4.0  3.0  2.0  1.0


```

默认情况下，那些列会从 DataFrame 中移除，但也可以将其保留下来：


```
In [63]: frame.set_index(['c', 'd'], drop=False)
Out[63]:
       a  b    c  d
c   d
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

```
reset_index 的功能跟 set_index 刚好相反，层次化索引的级别会被转移到列里面：


```
In [64]: frame2
Out[64]:
       a  b
c   d
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [65]: frame2.reset_index()
Out[65]:
     c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1


```

## 8.2 合并数据集

pandas 对象中的数据可以通过一些方式进行合并：

-  pandas.merge 可根据一个或多个键将不同 DataFrame 中的行连接起来。SQL 或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的 join 操作。
-  pandas.concat 可以沿着一条轴将多个对象堆叠到一起。
- 实例方法 combine_first 可以将重复数据编接在一起，用一个对象中的值填充另一个对象中的缺失值。

我将分别对它们进行讲解，并给出一些例子。本书剩余部分的示例中将经常用到它们。

## 数据库风格的 DataFrame 合并

数据集的合并（merge）或连接（join）运算是通过一个或多个键将行链接起来的。这些运算是关系型数据库（基于SQL）的核心。 pandas 的 merge 函数是对数据应用这些算法的主要切入点。

以一个简单的例子开始：

```
In [35]: df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
   ....:                     'data1': range(7)})

In [36]: df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
   ....:                     'data2': range(3)})

In [37]: df1
Out[37]: 
   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b

In [38]: df2
Out[38]: 
   data2 key
0      0   a
1      1   b
2      2   d


```

这是一种多对一的合并。df1 中的数据有多个被标记为 a 和 b 的行，而 df2 中 key 列的每个值则仅对应一行。对这些对象调用 merge 即可得到：

```
In [70]: pd.merge(df1, df2)
Out[70]:
   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

# 第 一列的索引， 先不用管，然后以 key 列为准，b,b,b,a,a,a 的顺序 调整其他列的顺序

```

注意，我并没有指明要用哪个列进行连接。如果没有指定，merge 就会将重叠列的列名当做键。不过，最好明确指定一下：

```
In [71]: pd.merge(df1, df2, on='key')
Out[71]:
   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0


```
如果两个对象的列名不同，也可以分别进行指定：


```
In [72]: df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b']
    ...: ,'data1':range(7)})

In [73]: df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})

In [74]: pd.merge(df3, df4, left_on='lkey', right_on='rkey')
Out[74]:
   data1 lkey  data2 rkey
0      0    b      1    b
1      1    b      1    b
2      6    b      1    b
3      2    a      0    a
4      4    a      0    a
5      5    a      0    a

In [75]: df3
Out[75]:
   data1 lkey
0      0    b
1      1    b
2      2    a
3      3    c
4      4    a
5      5    a
6      6    b

In [76]: df4
Out[76]:
   data2 rkey
0      0    a
1      1    b
2      2    d


```
- 可能你已经注意到了，结果里面 c 和 d 以及与之相关的数据消失了。默认情况下，merge 做的是“内连接”；结果中的键是**交集**。

- 其他方式还有"left"、"right"以及"outer"。**外连接求取的是键的并集**，组合了左连接和右连接的效果：

```
In [77]: pd.merge(df1, df2, how='outer')
Out[77]:
   data1 key  data2
0    0.0   b    1.0
1    1.0   b    1.0
2    6.0   b    1.0
3    2.0   a    0.0
4    4.0   a    0.0
5    5.0   a    0.0
6    3.0   c    NaN
7    NaN   d    2.0


```
表8-1对这些选项进行了总结。

![表8-1 不同的连接类型](./images/8_1.png)

多对多的合并有些不直观。看下面的例子：

```
In [78]: df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'dat
    ...: a1':range(6)})

In [79]: df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],'data2':r
    ...: ange(5)})

In [80]: df1
Out[80]:
   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b

In [81]: df2
Out[81]:
   data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d

In [82]: pd.merge(df1, df2, on='key', how='left')
Out[82]:
    data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0
4       2   a    0.0
5       2   a    2.0
6       3   c    NaN
7       4   a    0.0
8       4   a    2.0
9       5   b    1.0
10      5   b    3.0


```
多对多连接产生的是行的笛卡尔积。由于左边的 DataFrame 有 3 个 "b" 行，右边的有 2 个，所以最终结果中就有 6 个 "b" 行。

理解： 以 b 为例， 就只看前两个 

```
data1  key
  0   b
  1   b
  
data2  key 
  1   b
  3   b
  
# 笛卡尔积 2 *2 =4

      data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0

```

连接方式只影响出现在结果中的不同的键的值：

```
In [83]: pd.merge(df1, df2, how='inner')
Out[83]:
   data1 key  data2
0      0   b      1
1      0   b      3
2      1   b      1
3      1   b      3
4      5   b      1
5      5   b      3
6      2   a      0
7      2   a      2
8      4   a      0
9      4   a      2


```
要根据多个键进行合并，传入一个由列名组成的列表即可：

```
In [51]: left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
   ....:                      'key2': ['one', 'two', 'one'],
   ....:                      'lval': [1, 2, 3]})

In [52]: right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
   ....:                       'key2': ['one', 'one', 'one', 'two'],
   ....:                       'rval': [4, 5, 6, 7]})

In [88]: left
Out[88]:
  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3

In [89]: right
Out[89]:
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [53]: pd.merge(left, right, on=['key1', 'key2'], how='outer')
Out[53]: 
  key1 key2  lval  rval
0  foo  one   1.0   4.0
1  foo  one   1.0   5.0
2  foo  two   2.0   NaN
3  bar  one   3.0   6.0
4  bar  two   NaN   7.0


```

结果中会出现哪些键组合取决于所选的合并方式，你可以这样来理解：多个键形成一系列元组，并将其当做单个连接键（当然，实际上并不是这么回事）。

>注意：在进行列－列连接时， DataFrame 对象中的索引会被丢弃。

对于合并运算需要考虑的最后一个问题是对重复列名的处理。虽然你可以手工处理列名重叠的问题（查看前面介绍的重命名轴标签），但 merge 有一个更实用的 suffixes 选项，用于指定附加到左右两个 DataFrame 对象的重叠列名上的字符串：

```
In [91]: pd.merge(left, right, on='key1')
Out[91]:
  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7

In [92]: left
Out[92]:
  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3

In [93]: right
Out[93]:
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7

# key2 的列明是重复的，默认合并后在末尾添加的是 _x ,_y

# 指定添加 后缀

In [94]: pd.merge(left, right, on='key1', suffixes=('_left', '_right'))
    ...:
Out[94]:
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


```
merge 的参数请参见表 8-2。使用 DataFrame 的行索引合并是下一节的主题。

表8-2 merge 函数的参数

![](./images/8_2_0.png)
![](./images/8_2_1.png)

indicator 添加特殊的列 `_merge`，它可以指明每个行的来源，它的值有 left_only、right_only 或 both，根据每行的合并数据的来源。

## 索引上的合并

有时候， DataFrame 中的连接键位于其索引中。在这种情况下，你可以传入 `left_index=True` 或 `right_index=True`（或两个都传）以说明索引应该被用作连接键：

```
In [95]: left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],'v
    ...: alue':range(6)})

In [96]: right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a', 'b']
    ...: )

In [97]: left1
Out[97]:
  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

In [98]: right1
Out[98]:
   group_val
a        3.5
b        7.0

In [99]: pd.merge(left1, right1, left_on='key', right_index=True)
Out[99]:
  key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0


```
由于默认的 `merge`方法是求取连接键的交集，因此你可以通过外连接的方式得到它们的并集：

```
In [101]: pd.merge(left1, right1, left_on='key', right_index=True, how=
     ...: 'outer')
Out[101]:
  key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN

```
对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并：


```
In [103]: lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
     ...:    ....:                                'Nevada', 'Nevada'],
     ...:    ....:                       'key2': [2000, 2001, 2002, 200
     ...: 1, 2002],
     ...:    ....:                       'data': np.arange(5.)})
     ...:

In [104]: righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
     ...:    ....:                       index=[['Nevada', 'Nevada', 'O
     ...: hio', 'Ohio',
     ...:    ....:                               'Ohio', 'Ohio'],
     ...:    ....:                              [2001, 2000, 2000, 2000
     ...: , 2001, 2002]],
     ...:    ....:                       columns=['event1', 'event2'])
     ...:
     ...:

In [105]:

In [105]: lefth
Out[105]:
   data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002

In [106]: righth
Out[106]:
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


```

这种情况下，你必须以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理）：

```
In [107]: pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index
     ...: =True)
Out[107]:
   data    key1  key2  event1  event2
0   0.0    Ohio  2000       4       5
0   0.0    Ohio  2000       6       7
1   1.0    Ohio  2001       8       9
2   2.0    Ohio  2002      10      11
3   3.0  Nevada  2001       0       1

In [108]: pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index
     ...: =True, how='outer')
Out[108]:
   data    key1  key2  event1  event2
0   0.0    Ohio  2000     4.0     5.0
0   0.0    Ohio  2000     6.0     7.0
1   1.0    Ohio  2001     8.0     9.0
2   2.0    Ohio  2002    10.0    11.0
3   3.0  Nevada  2001     0.0     1.0
4   4.0  Nevada  2002     NaN     NaN
4   NaN  Nevada  2000     2.0     3.0



```
同时使用合并双方的索引也没问题：

```
In [109]: left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index =
     ...: ['a', 'c', 'e'], columns=['Ohio', 'Nevada'])

In [110]: right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 1
     ...: 4]], index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabam
     ...: a'])

In [111]: left2
Out[111]:
   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0


In [113]: right2
Out[113]:
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0

In [115]: pd.merge(left2, right2, how='outer', left_index=True, right_i
     ...: ndex=True)
Out[115]:
   Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0


```
 DataFrame 还有一个便捷的join实例方法，它能更为方便地实现按索引合并。它还可用于合并多个带有相同或相似索引的 DataFrame 对象，但要求没有重叠的列。在上面那个例子中，我们可以编写：

```


```

因为一些历史版本的遗留原因， DataFrame 的join方法默认使用的是左连接，保留左边表的行索引。它还支持在调用的 DataFrame 的列上，连接传递的 DataFrame 索引：

```


```
最后，对于简单的索引合并，你还可以向join传入一组 DataFrame ，下一节会介绍更为通用的concat函数，也能实现此功能：

```


```
## 轴向连接

另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。NumPy的 concatenation 函数可以用 NumPy 数组来做：


```


```
对于 pandas 对象（如Series和 DataFrame ），带有标签的轴使你能够进一步推广数组的连接运算。具体点说，你还需要考虑以下这些东西：

- 如果对象在其它轴上的索引不同，我们应该合并这些轴的不同元素还是只使用交集？
- 连接的数据集是否需要在结果对象中可识别？
- 连接轴中保存的数据是否需要保留？许多情况下， DataFrame 默认的整数标签最好在连接时删掉。

 pandas  的 concat 函数提供了一种能够解决这些问题的可靠方式。我将给出一些例子来讲解其使用方式。假设有三个没有重叠索引的 Series：


```


```
对这些对象调用concat可以将值和索引粘合在一起：


```


```

默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。如果传入axis=1，则结果就会变成一个 DataFrame （axis=1是列）：

```


```

这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。传入join='inner'即可得到它们的交集：

```


```
在这个例子中，f和g标签消失了，是因为使用的是join='inner'选项。

你可以通过join_axes指定要在其它轴上使用的索引：



```


```

不过有个问题，参与连接的片段在结果中区分不开。假设你想要在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的：


```


```
如果沿着axis=1对Series进行合并，则keys就会成为 DataFrame 的列头：

```


```
同样的逻辑也适用于 DataFrame 对象：

```


```
如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值：

```


```
此外还有两个用于管理层次化索引创建方式的参数（参见表8-3）。举个例子，我们可以用names参数命名创建的轴级别：


```


```
最后一个关于 DataFrame 的问题是， DataFrame 的行索引不包含任何相关数据：


```


```

在这种情况下，传入ignore_index=True即可：

```


```
![表8-3 concat函数的参数](./images/8_3.png)

## 合并重叠数据

还有一种数据组合问题不能用简单的合并（merge）或连接（concatenation）运算来处理。比如说，你可能有索引全部或部分重叠的两个数据集。举个有启发性的例子，我们使用NumPy的where函数，它表示一种等价于面向数组的if-else：


```


```
Series有一个combine_first方法，实现的也是一样的功能，还带有 pandas 的数据对齐：

```


```
对于 DataFrame ，combine_first自然也会在列上做同样的事情，因此你可以将其看做：用传递对象中的数据为调用对象的缺失数据“打补丁”：

```


```
## 8.3 重塑和轴向旋转

有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。

### 重塑层次化索引

层次化索引为 DataFrame 数据的重排任务提供了一种具有良好一致性的方式。主要功能有二：

- stack：将数据的列“旋转”为行。
-  unstack ：将数据的行“旋转”为列。

我将通过一系列的范例来讲解这些操作。接下来看一个简单的 DataFrame ，其中的行列索引均为字符串数组：


```


```
对该数据使用stack方法即可将列转换为行，得到一个Series：

```


```

对于一个层次化索引的Series，你可以用 unstack 将其重排为一个 DataFrame ：

```


```
默认情况下， unstack 操作的是最内层（stack也是如此）。传入分层级别的编号或名称即可对其它级别进行 unstack 操作：


```


```
如果不是所有的级别值都能在各分组中找到的话，则 unstack 操作可能会引入缺失数据：


```


```
stack默认会滤除缺失数据，因此该运算是可逆的：


```


```
在对 DataFrame 进行 unstack 操作时，作为旋转轴的级别将会成为结果中的最低级别：


```


```
当调用stack，我们可以指明轴的名字：

```


```

## 将“长格式”旋转为“宽格式”

多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。我们先加载一些示例数据，做一些时间序列规整和数据清洗：


```


```

这就是多个时间序列（或者其它带有两个或多个键的可观察数据，这里，我们的键是date和item）的长格式。表中的每行代表一次观察。

关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有一个好处：随着表中数据的添加，item列中的值的种类能够增加。在前面的例子中，date和item通常就是主键（用关系型数据库的说法），不仅提供了关系完整性，而且提供了更为简单的查询支持。有的情况下，使用这样的数据会很麻烦，你可能会更喜欢 DataFrame ，不同的item值分别形成一列，date列中的时间戳则用作索引。 DataFrame 的pivot方法完全可以实现这个转换：


```


```
前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充 DataFrame 的数据列。假设有两个需要同时重塑的数据列：

```


```
如果忽略最后一个参数，得到的 DataFrame 就会带有层次化的列：

```


```
注意，pivot其实就是用 set_index 创建层次化索引，再用 unstack 重塑：


```


```
## 将“宽格式”旋转为“长格式”

旋转 DataFrame 的逆运算是 pandas .melt。它不是将一列转换到多个新的 DataFrame ，而是合并多个列成为一个，产生一个比输入长的 DataFrame 。看一个例子：


```


```

key列可能是分组指标，其它的列是数据值。当使用 pandas .melt，我们必须指明哪些列是分组指标。下面使用key作为唯一的分组指标：

```


```
使用pivot，可以重塑回原来的样子：


```


```
因为pivot的结果从列创建了一个索引，用作行标签，我们可以使用re set_index 将数据移回列：

```


```
你还可以指定列的子集，作为值的列：

```


```

 pandas .melt也可以不用分组指标：

```

```


## 8.4 总结

现在你已经掌握了 pandas 数据导入、清洗、重塑，我们可以进一步学习matplotlib数据可视化。我们在稍后会回到 pandas ，学习更高级的分析。